In [ ]:
import requests
import json
import pandas as pd

In [ ]:
df_foi = pd.read_excel('percorso_file.xlsx', sheet_name='FeatureOfInterests')
df_csv_foi = df_foi.to_csv('percorso_file.csv', index=False)

df_csv_foi = pd.read_csv('percorso_file.csv')
df_csv_foi = df_csv_foi.sort_values(by='unique Identifier\n(M)')
df_csv_foi.head()

In [ ]:
print(df_csv_foi.columns)

In [ ]:
#BATCH

json_struct = []
iot_id_counter = 1

fixed_part = {
    "POST to": "v1.1/$batch",
    "Headers": "Content-Type: application/json",
    "requests": []
}


In [ ]:
def handle_nan(value):
    # Converte NaN numerici e stringhe 'nan' in Null
    if pd.isna(value) or value in ["nan", "NaN", None, np.nan]:
        return None
    return value

In [3]:
# Vilnius
# polygon = [[[25.247037, 54.6957306], [25.2908125, 54.6957306], [25.2908125, 54.6637075], [25.247037, 54.6637075], [25.247037, 54.6957306]]]
#"feature": {
    #"type": "Polygon",
    #"coordinates": polygon
  #},

In [ ]:
def parse_point(longitudes, latitudes):
    
    try:
        # Converti le coordinate in liste di numeri
        longitudes = [float(l.replace(",", ".")) for l in longitudes.split()]
        latitudes = [float(l.replace(",", ".")) for l in latitudes.split()]
        
        return [longitudes[0], latitudes[0]]  # Restituisce il primo punto

    except ValueError:
        return None  
    
    return None  # Se non ci sono dati validi


In [ ]:
for index, row in df_csv_foi.iterrows():
    json_data = {
        "id": str(row["unique Identifier\n(M)"]),
        "method": "post",
        "url": "FeaturesOfInterest",
        "body": {
            "name": handle_nan(str(row["name \n(M)"])),  # Rimuove stringhe 'nan'
            "description": handle_nan(str(row["description \n(M)"])),
            "encodingType": "application/geo+json",
            "feature": {
                "type": "Point",
                "coordinates":     "coordinates": parse_point(row["Longitude coordinate (O)"], row["Latitude coordinate (O)"])

            },
            "properties": {
                "identifier": str(row["unique Identifier\n(M)"]),
                "localIdentifier": str(row["local Identifier\n(M)"]),
                "room": {
                    "indoorHeightOfTheRoom": "",
                    "levelFloor": handle_nan(row["level / floor\n(O)"]),
                    "useOfTheSingleRoom": handle_nan(str(row["use of the single room (or building part) - (O)"])),
                    "useDetails": handle_nan(str(row["use_details\n(O)"])),
                    "floorArea": handle_nan(row["floor area\n(O)"]),
                    "glazedArea": handle_nan(row["glazed area\n(O)"]),
                    "ventilation": handle_nan(str(row["ventilation\n(O)"]))
                },
                "occupants": {
                    "numberOfOccupants": handle_nan(row["number of occupants\n(O)"]),
                    "mainTypeOfOccupants": handle_nan(str(row["main type of occupants\n(O)"])),
                    "genderOfOccupants": handle_nan(str(row["gender of occupants\n(O)"])),
                    "averageAgeOfOccupants": handle_nan(str(row["average age of occupants\n(O)"])),
                    "occupancyHoursMon": handle_nan(str(row["Occupancy hours\nMon (O)"])),
                    "occupancyHoursTue": handle_nan(str(row["Occupancy hours\nTue (O)"])),
                    "occupancyHoursWed": handle_nan(str(row["Occupancy hours\nWed (O)"])),
                    "occupancyHoursThu": handle_nan(str(row["Occupancy hours\nThu (O)"])),
                    "occupancyHoursFri": handle_nan(str(row["Occupancy hours\nFri (O)"])),
                    "occupancyHoursSat": handle_nan(str(row["Occupancy hours\nSat (O)"])),
                    "occupancyHoursSun": handle_nan(str(row["Occupancy hours\nSun (O)"]))
                }
            }
        }
    }
    fixed_part["requests"].append(json_data)

    iot_id_counter += 1

In [ ]:
formatted_json = json.dumps(fixed_part, indent=2, ensure_ascii=False)

try:
    json.loads(formatted_json)
    print("Il JSON è stato formattato correttamente.")
except json.JSONDecodeError as e:
    print(f"Errore nella formattazione: {e}")
    
with open('', 'w', encoding='utf-8') as json_file:
    json_file.write(formatted_json)
